In [1]:
import cv2
import mediapipe as mp
import time
import math

class HandDetector:
    
    def __init__(self,mode = False, maxHands = 2, complexity = 1, detectionCon = 0.5, trackCon = 0.5):
        
        self.mode = mode
        self.maxHands = maxHands
        self.complexity = complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHands = mp.solutions.hands
        
        self.Hands = self.mpHands.Hands(self.mode,self.maxHands,self.complexity,
                                        self.detectionCon,self.trackCon)
        
        self.tipIds = [4,8,12,16,20]
        
    def findHands(self,image,draw = True,flipType = True):
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.Hands.process(imageRGB)
        allHands = []
        h,w,c = image.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                # Hands Type --- > "Right" or "Left"
                #print(handType,handLms)
                myHand = {}
                mylmList = []
                xList = []
                yList = []
                
                for id,lm in enumerate(handLms.landmark):
                    px,py,pz = int(lm.x*w),int(lm.y*h),int(lm.z*w)
                    mylmList.append([px,py,pz])
                    xList.append(px)
                    yList.append(py)
                    
                xmin,xmax = min(xList),max(xList)
                ymin,ymax = min(yList),max(yList)
                boxW,boxH = xmax - xmin, ymax-ymin
                bbox = xmin,ymin,xmax,ymax
                
                cx,cy = bbox[0] + (boxW // 2), bbox[1] + (boxH // 2)
                
                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx,cy)
                
                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                else:
                    myHand["type"] = handType.classification[0].label
                
                # Bütün dataları aldık !!! 
                allHands.append(myHand)
                
                if draw:
                    
                    self.mpDraw.draw_landmarks(image,handLms,self.mpHands.HAND_CONNECTIONS)
                    
                    cv2.rectangle(image,(bbox[0]-20,bbox[1]-20),(bbox[2]+20, bbox[3] + 20),(255,0,255),2)
                    
                    cv2.putText(image,myHand["type"],(bbox[0] - 30, bbox[1] - 30), cv2.FONT_HERSHEY_PLAIN,
                               2,(255,0,255),2)
        if draw:
            return allHands,image
        else:
            return allHands
        
    def fingersUp(self,myHand):
        
        myHandType = myHand["type"]
        mylmList = myHand["lmList"]
        
        if self.results.multi_hand_landmarks:
            
            fingers = []
            
            if myHandType == "Right":
                
                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
            else:
                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
            for id in range(1,5):
                if mylmList[self.tipIds[id]][1] < mylmList[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)
        
        return fingers
    
    def findDistance(self,p1,p2,image = None):
        
        x1,y1 = p1
        x2,y2 = p2
        cx,cy = (x1+x2) // 2, (y1+y2) // 2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)
        if image is not None:
            cv2.circle(image,(x1,y1),15,(255,0,255),cv2.FILLED)
            cv2.circle(image,(x2,y2),15,(255,0,255),cv2.FILLED)
            cv2.line(image,(x1,y1),(x2,y2),(255,0,255),3)
            cv2.circle(image,(cx,cy),15,(255,0,255),cv2.FILLED)
            return length,info,image
        else:
            return length,info

In [7]:
pTime = 0
def fps():
    
    global pTime
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps

def main():
    
    cap = cv2.VideoCapture(0)
    
    hand_detector = HandDetector(detectionCon=0.8,maxHands=2)
    
    while True:
        
        conn, frame = cap.read()
        
        allHands, frame = hand_detector.findHands(frame)
        
        if allHands:
            #print(allHands)
            hands = allHands[0]
            lmList1 = hands["lmList"]
            bbox1 = hands["bbox"]
            centerpoint1 = hands["center"]
            handType1 = hands["type"]
            fingers1 = hand_detector.fingersUp(hands)
            #print(fingers1)
            length,info,frame = hand_detector.findDistance(lmList1[4][:-1],lmList1[8][:-1],frame)
            
            if len(allHands) == 2:
                #print(allHands)
                hands2 = allHands[1]
                lmList2 = hands2["lmList"]
                bbox2 = hands2["bbox"]
                centerpoint2 = hands2["center"]
                handType2 = hands2["type"]
                fingers2 = hand_detector.fingersUp(hands2)
                #print(fingers2)
                length,info,frame = hand_detector.findDistance(lmList1[4][:-1],lmList2[4][:-1],frame)
        
        
        fps_ = fps()
        cv2.putText(frame,f"FPS : {int(fps_)}",(20,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
        cv2.imshow("Frame",frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
             break
                
    cap.release()
    cv2.destroyAllWindows()
    
if __name__ == "__main__":
    main()

In [55]:
#%%capture
#!pip install applescript
import applescript
applescript.run("set volume output volume 50",background=True)

In [2]:
import cv2
import time
import numpy as np
import math
import applescript
wCam,hCam = 1280,720

cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)

video_writer = cv2.VideoWriter("volume.mp4",cv2.VideoWriter_fourcc(*"XVID"),30,(wCam,hCam))

pTime = 0
def fps():
    
    global pTime
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps

hand_detector = HandDetector(maxHands=1,detectionCon=0.7,complexity=0)
volBar = 400
vol = 0

while True:
    conn,frame = cap.read()
    
    allHands,frame = hand_detector.findHands(frame,draw = True,flipType = True)
    
    if allHands:
        
        hands1 = allHands[0]
        lmList1 = hands1["lmList"]
        bbox1 = hands1["bbox"]
        centerpoint1 = hands1["center"]
        handType1 = hands1["type"]
        #print(len(lmList1))
        #print(lmList1[4],lmList1[8])
        
        x1,y1 = lmList1[4][0],lmList1[4][1]
        x2,y2 = lmList1[8][0],lmList1[8][1]
        cx,cy = (x1+x2) // 2, (y1+y2) // 2
        
        cv2.circle(frame,(centerpoint1[0],centerpoint1[1]),25,(255,255,255),cv2.FILLED)
        
        cv2.circle(frame,(x1,y1),15,(255,0,255),cv2.FILLED)
        cv2.circle(frame,(x2,y2),15,(255,0,255),cv2.FILLED)
        cv2.circle(frame,(cx,cy),15,(255,0,255),cv2.FILLED)
        cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),3)
        
        length = math.hypot(x2-x1,y2-y1)
        #print(length)
        vol = np.interp(length,[50,300],[0,100])      
        volBar = np.interp(length,[50,300],[400,150])      
        
        string = "set volume output volume " + str(vol)
        applescript.run(string)
        
        #print(int(length),vol)
        
        if length < 50:
            cv2.circle(frame,(cx,cy),15,(0,255,0),cv2.FILLED)
        
    cv2.rectangle(frame,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(frame,(50,int(volBar)),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(frame,f"Volume : {int(vol)}%",(20,120),cv2.FONT_HERSHEY_PLAIN,2,(0,255,0),2)
    
    fps_ = fps()
    cv2.putText(frame,f"FPS : {int(fps_)}",(20,70),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),3)
    cv2.imshow("Frame",frame)
    video_writer.write(frame)
    
    if cv2.waitKey(1) & 0xff == ord("q"):
        break

video_writer.release()
cap.release()
cv2.destroyAllWindows()